In [1]:
!pip install -r requirements.txt

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from __future__ import print_function
import torch
from torchvision import datasets, transforms
import numpy as np
from WeakLearners import WongNeuralNetCIFAR10
import matplotlib.pyplot as plt


In [3]:
from Boosting import runMemlessCifarBoosting

In [4]:
import utils

In [5]:
cuda = torch.device('cuda:0')

In [6]:
%load_ext autoreload
%autoreload 2

## Vanilla PreactResNet

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size=100, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(
    testset, batch_size=300, shuffle=True, num_workers=2)

In [ ]:
cifar10_vanilla = WongNeuralNetCIFAR10()
cifar10_vanilla.fit(train_loader, test_loader, alpha = 0, epochs = 1, adv=False, epsilon=0)

In [ ]:
cifar10_vanilla.plot_val_accuracies()

In [ ]:
print(len(cifar10_vanilla.iters), len(cifar10_vanilla.val_accuracies))

## Experiments for CIFAR10 boosting

In [7]:
# maxIt_vals = [10, 20, 30, 50, 100]
# maxIt_vals = [10, 50, 100, 500]
maxIt_vals = [200, 500]
# default maxIt for weak learner is infinity (i.e. runs each epoch on the entire batch)

In [8]:
num_wl = 100 # maybe around 100? #later: maybe change this to an array?
num_epochs = 1

In [9]:
all_val_accuracies = []
all_wl = []
all_wl_weights = []

In [ ]:
from pytorch_memlab import LineProfiler
from Boosting import SchapireWongMulticlassBoostingMemoryLess
with LineProfiler(runMemlessCifarBoosting, SchapireWongMulticlassBoostingMemoryLess) as prof:
    for maxIt in maxIt_vals:
        wl, wl_weights, val_accuracies = runMemlessCifarBoosting(num_wl, maxIt, num_epochs)
        all_val_accuracies.append(val_accuracies)
        all_wl.append(wl)
        all_wl_weights.append(wl_weights)
prof.display()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------------------------------------------
Training 0th weak learning
Files already downloaded and verified
adv: False
maxIt: 200
Iteration:  1

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/cifar.py:118: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  img, target = self.data[index], self.targets[index]



memory usage: 143831040
Iteration:  11
memory usage: 143834624
Iteration:  21
memory usage: 143838208
Iteration:  31
memory usage: 143841792
Iteration:  41
memory usage: 143845376
Iteration:  51
memory usage: 143848960
Iteration:  61
memory usage: 143852544
Iteration:  71
memory usage: 143856128
Iteration:  81
memory usage: 143859712
Iteration:  91
memory usage: 143863296
Iteration:  101
memory usage: 143866880
Iteration:  111
memory usage: 143870464
Iteration:  121
memory usage: 143874048
Iteration:  131
memory usage: 143877632
Iteration:  141
memory usage: 143881216
Iteration:  151
memory usage: 143884800
Iteration:  161
memory usage: 143888384
Iteration:  171
memory usage: 143891968
Iteration:  181
memory usage: 143895552
Iteration:  191
memory usage: 143899136


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:893: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


targets: tensor([6, 9, 9,  ..., 9, 1, 1])
Full Accuracy:  0.42572
Alpha:  0.37908331158049813
After newest WL score is:  0.41
----------------------------------------------------------------------------------------------------
Training 1th weak learning
Files already downloaded and verified
adv: False
maxIt: 200
Iteration:  1
memory usage: 280757760
Iteration:  11
memory usage: 280761344
Iteration:  21
memory usage: 280764928
Iteration:  31
memory usage: 280768512
Iteration:  41
memory usage: 280772096
Iteration:  51
memory usage: 280775680
Iteration:  61
memory usage: 280779264
Iteration:  71
memory usage: 280782848
Iteration:  81
memory usage: 280786432
Iteration:  91
memory usage: 280790016
Iteration:  101
memory usage: 280793600
Iteration:  111
memory usage: 280797184
Iteration:  121
memory usage: 280800768
Iteration:  131
memory usage: 280804352
Iteration:  141
memory usage: 280807936
Iteration:  151
memory usage: 280811520
Iteration:  161
memory usage: 280815104
Iteration:  171
m

In [ ]:
# TODO: plot the accuracies for each usage of maxIt
legend_labels = []
for i in range(len(maxIt_vals)):
    maxIt = maxIt_vals[i]
    val_accuracies = all_val_accuracies[i]
    num_wl = len(all_wl[i])
    plt.plot(np.arange(maxIt, num_wl * maxIt + 1, maxIt), val_accuracies)
    legend_labels.append(f"MaxIt: {maxIt}")
plt.legend(legend_labels)
plt.show()
    

In [ ]:
import csv

for i in range(len(maxIt_vals)):
    maxIt = maxIt_vals[i]
    wl = all_wl[i]
    wl_weights = all_wl_weights[i]
    val_accuracies = all_val_accuracies[i]
    wl_file = f"./results/wl_maxIt_{maxIt}.csv"
    wl_weights_file = f"./results/wl_weights_maxIt_{maxIt}.csv"
    val_acc_file = f"./results/val_accuracy_{maxIt}.csv"
    with open(wl_file, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(wl)
    with open(wl_weights_file, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(wl_weights)
    with open(val_acc_file, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(val_accuracies)
    

In [ ]:
# Potential idea: maybe compare wall-clock time b/w CIFAR10 boosting and regular nn training to reach a certain accuracy threshold?

## Compare with other benchmarks? (i.e. other pretrained ensembles?)